<a href="https://colab.research.google.com/github/s-Arathy/star.sh/blob/main/acepath3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U kaleido

In [ ]:
!pip install pdfplumber
!pip install joblib


In [ ]:
!pip install pyngrok
!pip install flask-cors

In [ ]:
from google.colab import files

# Upload CSV and PDF files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib  # For saving the trained model

# Load your CSV data
csv_file_path = 'train.csv'  # Path to your training data
data = pd.read_csv(csv_file_path)

X = data['eng']
y = data['chapter']

# Vectorize the text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X)

# Train the classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, y)

# Save the vectorizer and the classifier model
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')  # Save TF-IDF vectorizer
joblib.dump(classifier, 'question_classifier.pkl')  # Save classifier


['question_classifier.pkl']

In [ ]:
from google.colab import files

# Upload CSV and PDF files
uploaded = files.upload()

Saving demotest.pdf to demotest (1).pdf


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2mhUQa2Zo1zfOTSbh9VH7L9GH58_6VEEhkAZ7AWp8r7RQTKie")



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib  # For saving the trained model


In [ ]:
from flask import Flask, jsonify, render_template
from pyngrok import ngrok
import pdfplumber
import re
import joblib  # For loading the saved model
from collections import Counter
import pandas as pd
import plotly.express as px
import base64

from flask_cors import CORS

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Step 1: Authenticate ngrok
ngrok.set_auth_token("2mhUQa2Zo1zfOTSbh9VH7L9GH58_6VEEhkAZ7AWp8r7RQTKie")  # Replace with your actual ngrok auth token

# Load the pre-trained model and vectorizer
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')  # Load TF-IDF vectorizer
classifier = joblib.load('question_classifier.pkl')  # Load pre-trained classifier

# Route for the root URL
@app.route('/')
def home():
    return "Welcome to the Question Classification API!"  # Simple message for the root URL

# Handle favicon requests
@app.route('/favicon.ico')
def favicon():
    return '', 204  # Return no content

# Function to extract text from the PDF
def extract_text_from_pdf(pdf_file_path):
    with pdfplumber.open(pdf_file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""  # Avoid NoneType errors
    return text

# Function to classify questions based on extracted text
def classify_questions(extracted_questions):
    tfidf_features = tfidf_vectorizer.transform(extracted_questions)
    predicted_chapters = classifier.predict(tfidf_features)
    return predicted_chapters

# Endpoint to generate the chart with the count of questions per chapter
@app.route('/chart', methods=['GET'])
def generate_chart():
    try:
        pdf_file_path = 'demotest.pdf'  # Path to your pre-uploaded PDF file

        # Step 1: Extract text from the PDF
        pdf_text = extract_text_from_pdf(pdf_file_path)

        # Step 2: Ensure the PDF text extraction is successful
        if not pdf_text:
            return jsonify({"error": "No text could be extracted from the PDF."}), 400

        # Step 3: Split the extracted text into questions
        extracted_questions = re.split(r'\d+\.\s', pdf_text)

        # Step 4: Ensure questions are extracted
        if not extracted_questions or len(extracted_questions) < 2:
            return jsonify({"error": "No questions could be extracted from the PDF."}), 400

        # Step 5: Classify the questions into chapters using the pre-trained model
        predicted_chapters = classify_questions(extracted_questions)

        # Step 6: Count the number of questions per chapter
        chapter_counts = Counter(predicted_chapters)

        # Step 7: Ensure chapters were predicted
        if not chapter_counts:
            return jsonify({"error": "No chapters predicted from the questions."}), 400

        # Step 8: Convert the counts to a DataFrame for easier plotting
        chapter_df = pd.DataFrame(list(chapter_counts.items()), columns=['Chapter', 'Count'])
        print(chapter_df)
        # Step 9: Generate the bar chart using Plotly
        fig = px.bar(chapter_df, x='Chapter', y='Count', title='Question Distribution by Chapter')
        print(fig)

        # Step 10: Convert the plot to a PNG image and encode it in base64

        img_bytes = fig.to_image(format="png")
        print(img_bytes)
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        '''img_base64= base64.b64encode(img_bytes.getvalue())
        result = img_base64
        return render_template('output.html',result=result)'''

        print("Base64 Image:", img_base64)  # Log the Base64 string for debugging




        # Step 11: Return the base64 encoded image in the response
        return jsonify({
            "chart": img_base64,
            "top_chapters": chapter_df.to_dict(orient='records')  # List of chapters with question counts
        }), 200

    except Exception as e:
        print(f"Error in generate_chart: {str(e)}")
        return jsonify({"error": str(e)}), 500

# Step 12: Start ngrok and run the Flask app
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)


Public URL: NgrokTunnel: "https://3be7-35-238-162-15.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 04:56:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 04:56:02] "GET /favicon.ico HTTP/1.1" 204 -


                                              Chapter  Count
0                                COMPARING QUANTITIES      1
1                                THE p-BLOCK ELEMENTS      9
2                                         PROBABILITY      1
3                               UNITS AND MEASUREMENT      2
4                                 CURRENT ELECTRICITY      5
5                                         GRAVITATION      4
6                           MOTION IN A STRAIGHT LINE      3
7                                   MOTION IN A PLANE      1
8                                      LAWS OF MOTION      4
9                              WORK, ENERGY AND POWER      5
10                         APPLICATION OF DERIVATIVES      1
11         SYSTEMS OF PARTICLES AND ROTATIONAL MOTION      3
12                                  STRUCTURE OF ATOM      5
13                          SURFACE AREAS AND VOLUMES      1
14                    MECHANICAL PROPERTIES OF FLUIDS      1
15                      

INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 04:56:10] "GET /chart HTTP/1.1" 200 -


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbc\x00\x00\x01\xf4\x08\x06\x00\x00\x00\xdfY\xfe\xdf\x00\x00 \x00IDATx^\xec\x9d\x05t\x1c\xc7\xb2\x86\x7f\x81Af\xc6\x98\x1d\xb3c&\xc9\x0c2333[\x86\x18d\x8e\x99\x99\x99\x99dff\x88\x991\xb1\xe3\x989\xb6%\xddS\xed\xbb\x9b\xd5jW;\xd4\x92V\xa99\'\xef>k\xbb{\xba\xbf\xee\x99\xf9\xa7\xa6\xba\xca%000\x10|0\x01&\xc0\x04\x98\x00\x13`\x02L\x80\t0\x81\x08J\xc0\x85\x05o\x04\x9dY\x1e\x16\x13`\x02L\x80\t0\x01&\xc0\x04\x98\x80 \xc0\x82\x97\x17\x02\x13`\x02L\x80\t0\x01&\xc0\x04\x98@\x84&\xc0\x827BO/\x0f\x8e\t0\x01&\xc0\x04\x98\x00\x13`\x02L\x80\x05/\xaf\x01&\xc0\x04\x98\x00\x13`\x02L\x80\t0\x81\x08M\x80\x05o\x84\x9e^\x1e\x1c\x13`\x02L\x80\t0\x01&\xc0\x04\x98\x00\x0b^^\x03L\x80\t0\x01&\xc0\x04\x98\x00\x13`\x02\x11\x9a\x00\x0b\xde\x08=\xbd<8&\xc0\x04\x98\x00\x13`\x02L\x80\t0\x01\x16\xbc\xbc\x06\x98\x00\x13`\x02L\x80\t0\x01&\xc0\x04"4\x01\x16\xbc\x11zzypL\x80\t0\x01&\xc0\x04\x98\x00\x13`\x02,xy\r0\x01&\xc0\x04\x98\x00\x13`\x02L\x80\tDh\x02,x#\xf4\xf4\xf2\xe0

INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 04:56:11] "GET /favicon.ico HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 05:04:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 05:04:25] "GET /favicon.ico HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 05:04:33] "GET /chart HTTP/1.1" 200 -


                                              Chapter  Count
0                                COMPARING QUANTITIES      1
1                                THE p-BLOCK ELEMENTS      9
2                                         PROBABILITY      1
3                               UNITS AND MEASUREMENT      2
4                                 CURRENT ELECTRICITY      5
5                                         GRAVITATION      4
6                           MOTION IN A STRAIGHT LINE      3
7                                   MOTION IN A PLANE      1
8                                      LAWS OF MOTION      4
9                              WORK, ENERGY AND POWER      5
10                         APPLICATION OF DERIVATIVES      1
11         SYSTEMS OF PARTICLES AND ROTATIONAL MOTION      3
12                                  STRUCTURE OF ATOM      5
13                          SURFACE AREAS AND VOLUMES      1
14                    MECHANICAL PROPERTIES OF FLUIDS      1
15                      

INFO:werkzeug:127.0.0.1 - - [29/Sep/2024 05:04:34] "GET /favicon.ico HTTP/1.1" 204 -
